In [1]:
import numpy as np
from collections import Counter

DEBUG = True

## Multiclass Classifier with Decision Tree using ID3 algorithm

### Utility functions

Min-Max normalization function

In [2]:
def normalize(matrix):
    """
    Min Max scaling
    matrix must be a np.ndarry
    """
    return (matrix - matrix.min(axis=0))/(matrix.max(axis=0) - matrix.min(axis=0))

Function to calculate Entropy

In [3]:
def entropy(y) -> float:
    m = len(y)
    class_freq_dict = Counter(y)
    entropy = -1 * sum([(class_count/m)*np.log2(class_count/m)
                        for class_count in class_freq_dict.values()])
    return entropy

Function to load csv file and return X features and y target matrix

In [4]:
def load_csv(file, unpack=True):
    data = np.genfromtxt(file, delimiter=',', encoding=None, dtype=str)
    X, y = data[:, :-1], data[:, -1].reshape(-1,1)
    return (np.array(X).astype(np.float), np.array(y)) if unpack else data

Function to randomly shuffle the array using the numpy.random.shuffle()

In [5]:
def shuffle_array(matrix, seed=None):
    """
    np.random.shuffle() shuffles a multidimensional arr using only the first dimension
    Alternative implementation of shuffle
    
    def shuffle_array(matrix):
        # shuffles an array in place based on the first dimension
        for i in range(len(matrix)):
            rd_idx = np.random.randint(i,len(matrix))
            matrix[i], matrix[rd_idx] = matrix[rd_idx], matrix[i] 
    """
    np.random.seed(seed)
    np.random.shuffle(matrix)

Function to generate k fold cross validation training and testing lists

In [6]:
def generate_kfold_train_test_set(X, y, test_frac=0.10):
    """
    Generates kfold train test sets from X_feat, y_target
    Returns X_train_list, y_train_list, X_test_list, y_test_list
    
    X must be np.array of ints or floats
    y is the target or label array
    """
    m = len(X)
    norm_X = normalize(X)
    test_data_count = int(test_frac*m)
    start, end = 0, test_data_count

    # sets to store features and labels of training and testing data
    X_train_list, y_train_list, X_test_list, y_test_list = [], [], [], []
    
    # 10-fold cross-validation:
    for _ in range(10):
        X_test = norm_X[start:end]
        X_train = np.concatenate([norm_X[:start], norm_X[end:]], axis=0)
        
        X_test_list.append(X_test)
        X_train_list.append(X_train)
        
        y_test = y[start:end].flatten()
        y_train = np.concatenate([y[:start], y[end:]], axis=0).flatten()
        
        y_test_list.append(y_test)
        y_train_list.append(y_train)
        
        # update test set start and end pointers
        start += test_data_count
        end += test_data_count

    # return the feature, label fold lists for both training and testing set.
    return X_train_list, y_train_list, X_test_list, y_test_list

Function to generate attr_dict

In [7]:
def get_attrib_dict_with_class_labels(X, y):
    """
    Builds a dict where,
        the keys are the feature idx, 
        the values are feature value and corresponding feature labels two-element lists
    """
    attr_dict = {}

    for feat_idx in range(X.shape[1]):
        feat_vals = X[:, feat_idx]
        attr_list = []
        
        for data_idx, val in enumerate(feat_vals):
            attr_list.append([val, y[data_idx]])

        attr_dict[feat_idx] = attr_list

    return attr_dict

Decision Tree Node Class

In [8]:
class Node:

    def __init__(self, class_label, left, right, threshold):
        self.class_label = class_label
        self.left = left
        self.right = right
        self.threshold = threshold

    def __repr__(self):
        return f"{self.class_label}, {self.left}, {self.right}, {self.threshold}"

Decision Tree Class

In [31]:
class DecisionTree():

    def __init__(self):
        self.root = None

    def get_major_class_label(self, class_values):
        try:
            return Counter(class_values).most_common(1)[0][0]
        except Exception as e:
            print(e)
            return None

    def get_best_threshold_perm_sorting(self, attri_list):
        """
        Calculates best threshold value for each feature
        attri_list are the values of the attr_dict
        WARNING: perm sorts the training data. Shuffling is discarded
        """
        attri_list.sort()
        orig_entropy = entropy([label for val, label in attri_list])
        max_info_gain = 0
        optimal_threshold = 0
        label_after_split_list = []
        optimal_left_idx_list, optimal_right_idx_list = [], []

        for i in range(len(attri_list)-1):
            if attri_list[i][1] == attri_list[i+1][1]:
                continue
                
            # candidate threshold is the midpoint between cur data and next data
            threshold = (attri_list[i][0] + attri_list[i+1][0])/2
            # lists that store index, value less than threshold
            lt_idx_list, lt_val_list = list(range(i+1)), [label for val, label in attri_list[:i+1]]
            # lists that store index, value greater than threshold
            gt_idx_list, gt_val_list = list(range(i+1,len(attri_list))), [label for val, label in attri_list[i+1:len(attri_list)]]

            # calculate the entropy of the "less" list
            entropy_lt_list = entropy(lt_val_list)
            # calculate the entropy of the "greater" list
            entropy_gt_list = entropy(gt_val_list)
            # calculate the info gain using the formula
            info_gain = orig_entropy - (
                entropy_lt_list*(len(lt_val_list)/len(attri_list)) +
                entropy_gt_list*(len(gt_val_list)/len(attri_list)))

            # if current info gain > max info gan
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                optimal_threshold = threshold
                optimal_left_idx_list = lt_idx_list
                optimal_right_idx_list = gt_idx_list

        return max_info_gain, optimal_threshold, optimal_left_idx_list, optimal_right_idx_list

    def get_best_threshold(self, attri_list):
        """
        Calculates best threshold value for each feature
        attri_list are the values of the attr_dict
        """

        # extract data vals and label vals from sorted attri_list
        data_list, class_label_list = map(list, zip(*sorted(attri_list)))

        orig_entropy = entropy(class_label_list)
        max_info_gain = 0
        optimal_threshold = 0
        label_after_split_list = []
        optimal_left_idx_list, optimal_right_idx_list = [], []

        for i in range(len(data_list)-1):
            # IMPORTANT speeding step. If the i and the i+1th class values are the same, don't split
            if class_label_list[i] == class_label_list[i+1]:
                continue

            # candidate threshold is the midpoint between cur data and next data
            threshold = (data_list[i] + data_list[i+1])/2
            # lists that store index, value less than threshold
            lt_idx_list, lt_val_list = [], []
            # lists that store index, value greater than threshold
            gt_idx_list, gt_val_list = [], []

            # for each index and value in attri_list
            for j, data_tuple in enumerate(attri_list):
                data, label = data_tuple
                # if value less or equal than the current theta:
                if data < threshold:
                    # update the "less" list of index and value
                    lt_idx_list.append(j)
                    lt_val_list.append(label)
                else:
                    # update the "greater" list of index and value
                    gt_idx_list.append(j)
                    gt_val_list.append(label)

            # calculate the entropy of the "less" list
            entropy_lt_list = entropy(lt_val_list)
            # calculate the entropy of the "greater" list
            entropy_gt_list = entropy(gt_val_list)
            # calculate the info gain using the formula
            info_gain = orig_entropy - (
                entropy_lt_list*(len(lt_val_list)/len(attri_list)) +
                entropy_gt_list*(len(gt_val_list)/len(attri_list)))

            # if current info gain > max info gan
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                optimal_threshold = threshold
                optimal_left_idx_list = lt_idx_list
                optimal_right_idx_list = gt_idx_list

        return max_info_gain, optimal_threshold, optimal_left_idx_list, optimal_right_idx_list

    def get_best_feature(self, attr_dict):
        """
        Method to select the optimum feature out of all the features.
        For each feature, get the optimum threshold and information gain
        """
        best_feat = None
        best_info_gain = -1
        best_threshold = 0
        best_left_idx_list = []
        best_right_idx_list = []

        # get max_info_gain using each feat as split to get the best_feat to split on
        for feat in attr_dict.keys():
            info_gain, threshold, left_idx_list, right_idx_list = self.get_best_threshold(
                attr_dict[feat])

            if info_gain > best_info_gain:
                best_feat = feat
                best_threshold = threshold
                best_info_gain = info_gain
                best_left_idx_list = left_idx_list
                best_right_idx_list = right_idx_list

        return [best_feat, best_threshold, best_left_idx_list, best_right_idx_list]

    def get_remainder_dict(self, attr_dict, index_split):
        split_dict = {}
        exclude_index_split = set(index_split)

        for feat in attr_dict.keys():
            class_label_list = []
            modified_list = []
            feat_val_list = attr_dict[feat]

            # feat_val_list = [[1.2,'iris-setosa'],[2.2,'iris-verisicolor'],....]
            for i, dl_tuple in enumerate(feat_val_list):
                if i in index_split:
                    class_label_list.append(dl_tuple[1])
                    modified_list.append(dl_tuple)
            split_dict[feat] = modified_list

        return split_dict, class_label_list

    def create_decision_tree(self, attr_dict, y_train, n_min_val):
        """
        Grow decision tree and return the root node
        """
        # if all the class labels are same, node is pure
        if len(set(y_train)) == 1:
            return Node(y_train[0], None, None, 0)
        # if num class vales are less than threshold, we assign the class with max values as the class label
        elif len(y_train) < n_min_val:
            max_value_class = self.get_major_class_label(y_train)
            return Node(max_value_class, None, None, 0)
        else:
            feat, threshold, left_idx_split, right_idx_split = self.get_best_feature(
                attr_dict)

            left_tree_dict, left_tree_class_labels = self.get_remainder_dict(
                attr_dict, left_idx_split)
            right_tree_dict, right_tree_class_labels = self.get_remainder_dict(
                attr_dict, right_idx_split)

            left_node = self.create_decision_tree(
                left_tree_dict, left_tree_class_labels, n_min_val)
            right_node = self.create_decision_tree(
                right_tree_dict, right_tree_class_labels, n_min_val)
            return Node(feat, left_node, right_node, threshold)

    def fit(self, X_train, y_train, eta_min_val):
        attr_dict = get_attrib_dict_with_class_labels(X_train, y_train)

        self.root = self.create_decision_tree(
            attr_dict, y_train, eta_min_val)

    def classify(self, data_row):
        cur_node = self.root
        #  while cur_node is not a leaf node
        while cur_node and (cur_node.left or cur_node.right):
            if data_row[cur_node.class_label] <= cur_node.threshold:
                cur_node = cur_node.left
            else:
                cur_node = cur_node.right
        return cur_node.class_label

    def predict(self, X):
        """
        Predicts labels for the X_feat data
        """
        return [self.classify(data_row) for data_row in X]

Evaluation of model

In [27]:
def get_accuracy(y_actual, y_pred):
    return len([1 for actual,pred in zip(y_actual, y_pred) if actual==pred])/len(y_pred)

In [28]:
def run_decision_trees(X, y, n_min):
    """
    function to run decision trees with k-Fold Cross validation
    """
    m, n = X.shape[0], X.shape[1]
    
    if isinstance(n_min, int):
        n_min_val = n_min          # Use values for node cutoff
    elif 0.0 < n_min < 1.0:
        n_min_val = round(n_min*m) # Use fractions for node cutoff
    else:
        raise ValueError("Fractions must be in range (0.0, 1.0)")
    
    # concatenate, shuffle and split  X,y
    concatenated_data = np.concatenate([X,y], axis=1)
    shuffle_array(concatenated_data)
    X,y = concatenated_data[:,:-1].astype('float'), concatenated_data[:,-1].reshape(-1,1)
    
    X_train_list, y_train_list, X_test_list, y_test_list = generate_kfold_train_test_set(
        X, y)

    decision_tree = DecisionTree()
    accuracy_list = []

    for i in range(10):
        X_train, y_train = X_train_list[i], y_train_list[i]
        X_test, y_test = X_test_list[i], y_test_list[i]

        # Fit the decision tree model
        decision_tree.fit(X_train, y_train, n_min_val)

        y_pred = decision_tree.predict(X_test)
        accuracy = get_accuracy(y_test, y_pred)

        accuracy_list.append(accuracy)
        print("Accuracy is ", "{:.4f}".format(accuracy))
    print("Average accuracy across 10-cross validation when cutoff is",
          n_min, "nodes is", "{:.4f}".format(np.mean(accuracy_list)))
    print("Standard deviation across 10-cross validation when cutoff is",
          n_min, "nodes is", "{:.4f}".format(np.std(accuracy_list)))

In [32]:
%%time
# n_min = minimum frac/num of examples for a node to become a leaf
# n_min_list_frac = [0.05,0.10,0.15,0.20,0.25]
n_min_list_val = [5, 10, 15, 20]
iris_data = "../data/iris.csv"
X,y = load_csv(iris_data)

for n_min in n_min_list_val:
    run_decision_trees(X, y, n_min)

Accuracy is  0.9333
Accuracy is  1.0000
Accuracy is  0.9333
Accuracy is  1.0000
Accuracy is  1.0000
Accuracy is  0.9333
Accuracy is  1.0000
Accuracy is  0.9333
Accuracy is  0.7333
Accuracy is  0.9333
Average accuracy across 10-cross validation when cutoff is 5 nodes is 0.9400
Standard deviation across 10-cross validation when cutoff is 5 nodes is 0.0757
Accuracy is  0.8667
Accuracy is  1.0000
Accuracy is  0.8667
Accuracy is  1.0000
Accuracy is  1.0000
Accuracy is  1.0000
Accuracy is  0.9333
Accuracy is  0.9333
Accuracy is  1.0000
Accuracy is  1.0000
Average accuracy across 10-cross validation when cutoff is 10 nodes is 0.9600
Standard deviation across 10-cross validation when cutoff is 10 nodes is 0.0533
Accuracy is  0.8000
Accuracy is  1.0000
Accuracy is  1.0000
Accuracy is  0.8667
Accuracy is  1.0000
Accuracy is  0.8667
Accuracy is  1.0000
Accuracy is  1.0000
Accuracy is  0.9333
Accuracy is  0.9333
Average accuracy across 10-cross validation when cutoff is 15 nodes is 0.9400
Standard

In [ ]:
%%time
# eta_min_list_val = [5, 10, 15, 20, 25]
eta_min_list_val = [5]
iris_data = "../data/spambase.csv"
X,y = load_csv(iris_data)

for eta_min in eta_min_list_val:
    run_decision_trees(X, y, eta_min)